In [1]:
import pandas as pd
import talib

In [20]:
df = pd.read_csv('all_dailies.csv')

In [21]:
df['value'] = df['close'] * df['volume']

In [22]:
latest = df[df['date'] == df['date'].max()].copy()
latest.sort_values('value', ascending=False, inplace=True)

In [26]:
order = latest['ticker'].tolist()

In [29]:
def ranking(ticker):
    if ticker not in order:
        return None
    return order.index(ticker)

In [37]:
order[:5]

['NVDA', 'SPY', 'QQQ', 'AMD', 'TSLA']

In [38]:
rankdict = dict(zip(order, list(range(len(order)))))

In [31]:
df['rank'] = df['ticker'].apply(ranking)

In [51]:
def altranking(ticker):
    if ticker not in rankdict:
        return None
    return int(rankdict[ticker])

In [54]:
df['altrank'] = df['ticker'].apply(altranking)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'function'

In [ ]:
df[['ticker', 'rank', 'altrank']].head(30)

,ticker,rank,altrank
0,A,468.0,468.0
1,AA,598.0,598.0
2,AAAU,1385.0,1385.0
3,AABA,NaN,NaN
4,AAC,NaN,NaN
5,AADR,9491.0,9491.0
6,AAL,277.0,277.0
7,AAMC,7424.0,7424.0
8,AAME,9557.0,9557.0
9,AAN,4804.0,4804.0


In [50]:
int(4.0)

4

In [47]:
df.head(30)

,ticker,volume,open,close,high,low,window_start,transactions,date,value,rank
0,A,783350,81.5700,81.6900,81.7100,80.5850,1554696000000000000,8040,2019-04-08,6.399186e+07,468.0
1,AA,2375490,29.5500,29.6600,29.7300,29.3200,1554696000000000000,13301,2019-04-08,7.045703e+07,598.0
2,AAAU,10896,13.0000,12.9600,13.0100,12.9600,1554696000000000000,29,2019-04-08,1.412122e+05,1385.0
3,AABA,16611754,73.6500,74.4600,74.7800,73.5800,1554696000000000000,78459,2019-04-08,1.236911e+09,NaN
4,AAC,97919,2.0400,2.0600,2.0700,1.9800,1554696000000000000,540,2019-04-08,2.017131e+05,NaN
5,AADR,6084,46.5000,46.5589,46.5589,46.3900,1554696000000000000,38,2019-04-08,2.832643e+05,9491.0
6,AAL,6219760,33.6400,33.8800,33.9529,33.4400,1554696000000000000,36850,2019-04-08,2.107255e+08,277.0
7,AAMC,1150,30.0900,30.0000,30.0900,30.0000,1554696000000000000,36,2019-04-08,3.450000e+04,7424.0
8,AAME,3206,2.6374,2.5200,2.6374,2.5200,1554696000000000000,15,2019-04-08,8.079120e+03,9557.0
9,AAN,298132,53.4000,53.5600,53.5700,52.6400,1554696000000000000,3414,2019-04-08,1.596795e+07,4804.0
